In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    


In [2]:
# Read Data
ECH_customer = pd.read_pickle('../data/pickle/ECH Customer Data.pickle')

df = ECH_customer
del ECH_customer

df.head()

,ecrid,name,city,Country ISO,Region,consortium,post_code,Classification
0,ECR-560156,National Institute for Research-Development in...,Timişoara,RO,NaN,nan,300569,Academic
1,ECR-10349400,Donostia International Physics Center,San Sebastian,ES,NaN,nan,20018,Government
2,ECR-29390542,Twist Bioscience,San Francisco,US,CA,nan,94158-2117,Corporate
3,ECR-10234417,University of Pennsylvania Department of Patho...,Philadelphia,US,PA,nan,19104,Academic
4,ECR-576739,Meerut Institute of Engineering and Technology,Meerut,IN,NaN,nan,250005,Academic


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown'], np.nan, inplace=True)
df

,ecrid,name,city,Country ISO,Region,consortium,post_code,Classification
0,ECR-560156,National Institute for Research-Development in...,Timişoara,RO,NaN,nan,300569,Academic
1,ECR-10349400,Donostia International Physics Center,San Sebastian,ES,NaN,nan,20018,Government
2,ECR-29390542,Twist Bioscience,San Francisco,US,CA,nan,94158-2117,Corporate
3,ECR-10234417,University of Pennsylvania Department of Patho...,Philadelphia,US,PA,nan,19104,Academic
4,ECR-576739,Meerut Institute of Engineering and Technology,Meerut,IN,NaN,nan,250005,Academic
...,...,...,...,...,...,...,...,...
137541,ECR-28176215,Quiron Bookstore,Tijuana,MX,NaN,nan,NaN,Corporate
137542,ECR-28176232,Library and Medical Equipment Canyon Brothers ...,Puebla,MX,NaN,nan,NaN,Government
137543,ECR-28176220,Educational and Cultural Promotion Francisco d...,Durango,MX,NaN,nan,NaN,Corporate
137544,ECR-28176240,Progress Trader and Consulting SA de CV,Naucalpan de Juárez,MX,NaN,nan,NaN,Corporate


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

ECR-24862456    1
ECR-461789      1
ECR-850846      1
ECR-219335      1
ECR-931143      1
               ..
ECR-546178      1
ECR-11027802    1
ECR-976731      1
ECR-11107803    1
ECR-969721      1
Name: ecrid, Length: 137546, dtype: int64
Retirement Home                                                                                      27
Institute of Nursing Education                                                                       17
KAISER PERMANENTE                                                                                    16
Departmental House of Solidarity                                                                     15
Social Medical Center                                                                                13
                                                                                                     ..
University of Texas Center for Cardiovascular Biology and Atherosclerosis Research                    1
Jinnah Medical and Dental Colleg

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
5,consortium,"[[nan, 1.0]]",2,float64,99.80
4,Region,"[[nan, CA, PA, TX, LA, UT, OH, OK, MD, NY, FL,...",69,object,61.28
7,Classification,"[[Academic, Government, Corporate, Distributio...",7,object,6.07
6,post_code,"[[300569, 20018, 94158-2117, 19104, 250005, na...",67750,object,5.09
3,Country ISO,"[[RO, ES, US, IN, LB, AT, CO, FR, CN, IT, JP, ...",208,object,0.03
0,ecrid,"[[ECR-560156, ECR-10349400, ECR-29390542, ECR-...",137546,object,0.00
1,name,[[National Institute for Research-Development ...,134400,object,0.00
2,city,"[[Timişoara, San Sebastian, San Francisco, Phi...",19104,object,0.00


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove other columns
df = df.drop(columns=[])

-------------------- Dropping Columns with missing data --------------------
Following 1 columns have missing data over the threshold and will be removed
Index(['consortium'], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
4,Region,"[[nan, CA, PA, TX, LA, UT, OH, OK, MD, NY, FL,...",69,object,61.28
6,Classification,"[[Academic, Government, Corporate, Distributio...",7,object,6.07
5,post_code,"[[300569, 20018, 94158-2117, 19104, 250005, na...",67750,object,5.09
3,Country ISO,"[[RO, ES, US, IN, LB, AT, CO, FR, CN, IT, JP, ...",208,object,0.03
0,ecrid,"[[ECR-560156, ECR-10349400, ECR-29390542, ECR-...",137546,object,0.00
1,name,[[National Institute for Research-Development ...,134400,object,0.00
2,city,"[[Timişoara, San Sebastian, San Francisco, Phi...",19104,object,0.00


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

Classification - Impute with most frequent

In [8]:
# #Filling all missing values with 0
# # data = data.fillna(0)

# #Filling missing values with the most frequest values
# df['Classification'].fillna(
#     df['Classification'].value_counts().idxmax(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = [])

# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
6,Classification,"[[Academic, Government, Corporate, Distributio...",7,object,6.07
4,Region,"[[nan, CA, PA, TX, LA, UT, OH, OK, MD, NY, FL,...",69,object,61.28


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
6,Classification,"[[Academic, Government, Corporate, Distributio...",7,object,6.07
4,Region,"[[nan, CA, PA, TX, LA, UT, OH, OK, MD, NY, FL,...",63,object,61.28


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 0 features - **********************


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
#print(columns_to_bin)

In [18]:
# Specify Columns to encode
# columns_to_exclude = []
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [19]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [20]:
#df.shape

In [21]:
df.head()

,ecrid,name,city,Country ISO,Region,post_code,Classification
0,ECR-560156,National Institute for Research-Development in...,Timişoara,RO,NaN,300569,Academic
1,ECR-10349400,Donostia International Physics Center,San Sebastian,ES,NaN,20018,Government
2,ECR-29390542,Twist Bioscience,San Francisco,US,CA,94158-2117,Corporate
3,ECR-10234417,University of Pennsylvania Department of Patho...,Philadelphia,US,PA,19104,Academic
4,ECR-576739,Meerut Institute of Engineering and Technology,Meerut,IN,NaN,250005,Academic


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = []

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calculate new variable activity_length

In [25]:
 df.head()

,ecrid,name,city,Country ISO,Region,post_code,Classification
0,ECR-560156,National Institute for Research-Development in...,Timişoara,RO,NaN,300569,Academic
1,ECR-10349400,Donostia International Physics Center,San Sebastian,ES,NaN,20018,Government
2,ECR-29390542,Twist Bioscience,San Francisco,US,CA,94158-2117,Corporate
3,ECR-10234417,University of Pennsylvania Department of Patho...,Philadelphia,US,PA,19104,Academic
4,ECR-576739,Meerut Institute of Engineering and Technology,Meerut,IN,NaN,250005,Academic


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = []

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,ecrid,name,city,Country ISO,Region,post_code,Classification
0,ECR-560156,National Institute for Research-Development in...,Timişoara,RO,NaN,300569,Academic
1,ECR-10349400,Donostia International Physics Center,San Sebastian,ES,NaN,20018,Government
2,ECR-29390542,Twist Bioscience,San Francisco,US,CA,94158-2117,Corporate
3,ECR-10234417,University of Pennsylvania Department of Patho...,Philadelphia,US,PA,19104,Academic
4,ECR-576739,Meerut Institute of Engineering and Technology,Meerut,IN,NaN,250005,Academic


Save clean pickle files

In [29]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['ech_file']+"_for_graphs.pickle")